## 1462. Course Schedule IV

There are a total of `numCourses` courses you have to take, labeled from 0 to `numCourses - 1`. You are given an array `prerequisites` where `prerequisites[i] = [ai, bi]` indicates that you must take course `ai` first if you want to take course `bi`.

For example, the pair `[0, 1]` indicates that you have to take course `0` before you can take course `1`.

Prerequisites can also be **indirect**. If course `a` is a prerequisite of course `b`, and course `b` is a prerequisite of course `c`, then course `a` is a prerequisite of course `c`.

You are also given an array `queries` where `queries[j] = [uj, vj]`. For the `jth` query, you should answer whether course `uj` is a prerequisite of course `vj` or not.

Return a boolean array `answer`, where `answer[j]` is the answer to the `jth` query.

### Example 1:
![](1462.0.png)

Input: `numCourses = 2, prerequisites = [[1,0]], queries = [[0,1],[1,0]]`
Output: `[false,true]`
Explanation: The pair `[1, 0]` indicates that you have to take course `1` before you can take course `0`.
Course `0` is not a prerequisite of course `1`, but the opposite is true.

### Example 2:
Input: `numCourses = 3, prerequisites = [[1,2],[1,0],[2,0]], queries = [[1,0],[1,2]]`
Output: `[true,true]`

### Example 3:
![](1462.1.png)

Input: `numCourses = 3, prerequisites = [[1,2],[1,0],[2,0]], queries = [[1,0],[1,2]]`
Output: `[true,true]`

In [39]:
from common import TestCase, Tester
from typing import List, Dict, Set
from collections import deque

class SolutionDfs:
    def dfs(self, adj: List[List[int]], visited: List[int], at: int, to_reach: int) -> bool:
        # Search is reachable from at
        visited[at] = True
        for to in adj[at]:
            if not visited[to]:
                self.dfs(adj, visited, to, to_reach)
        return visited[to_reach]

    def checkIfPrerequisite(self, numCourses: int, prerequisites: List[List[int]], queries: List[List[int]]) -> List[bool]:
        check_list = []
        adj = [[] for _ in range(numCourses)]
        for edge in prerequisites:
            adj[edge[0]].append(edge[1])
        for query in queries:
            visited = [False for _ in range(numCourses)]
            check_list.append(self.dfs(adj, visited, query[0], query[1]))
            
        return check_list

In [26]:

class SolutionDfsMemo:
    def dfs(self, adj: List[List[int]], visited: List[int], at: int, to_reach: int, memo: Dict[int, bool]) -> bool:
        if at == to_reach:
            return True
        if at in memo:
            return memo[at]
        visited[at] = True
        for to in adj[at]:
            if not visited[to]:
                if self.dfs(adj, visited, to, to_reach, memo):
                    memo[at] = True
                    return True
        memo[at] = False
        return False

    def checkIfPrerequisite(self, numCourses: int, prerequisites: List[List[int]], queries: List[List[int]]) -> List[bool]:
        check_list = []
        adj = [[] for _ in range(numCourses)]
        for edge in prerequisites:
            adj[edge[0]].append(edge[1])
        for query in queries:
            visited = [False for _ in range(numCourses)]
            memo = {}
            check_list.append(self.dfs(adj, visited, query[0], query[1], memo))
            
        return check_list


In [29]:
class SolutionFloydWarshall:
    def checkIfPrerequisite(self, numCourses: int, prerequisites: List[List[int]], queries: List[List[int]]) -> List[bool]:
        # Setup a reachable list 
        reachable = [[False] * numCourses for _ in range(numCourses)]
        
        # 
        for i, j in prerequisites:
            reachable[i][j] = True
        
        for k in range(numCourses):
            for i in range(numCourses):
                for j in range(numCourses):
                    if reachable[i][k] and reachable[k][j]:
                        reachable[i][j] = True
        
        return [reachable[i][j] for i, j in queries]

In [38]:
class SolutionTopSort:             
    
    def checkIfPrerequisite(self, numCourses: int, prerequisites: List[List[int]], queries: List[List[int]]) -> List[bool]:
        check_list = []
        adj = [[] for _ in range(numCourses)]
        for edge in prerequisites:
            adj[edge[0]].append(edge[1])
        
        # Use a set to store reachable nodes for each node
        reachable: List[Set[int]] = [set() for _ in range(numCourses)]
        
        # Get number of in degree for every vertex
        in_degree = [0 for _ in range(numCourses)]
        for i in range(numCourses):
            for vertex in adj[i]:
                in_degree[vertex] += 1
        
        # Initilize queue
        q = deque()
        for i in range(numCourses):
            if in_degree[i] == 0:
                q.append(i)
        
        # Initiate index       
        while q:
            at = q.popleft()
            for to in adj[at]:
                reachable[to].update(reachable[at])
                reachable[to].add(at)
                in_degree[to] -= 1
                if in_degree[to] == 0:
                    q.append(to)
        
        print(reachable)
        return [i in reachable[j] for i, j in queries]

In [37]:
test_cases = [
    TestCase((2, [[0,1]], [[0,1]]), [True]),
    TestCase((2, [], [[1,0]]), [False]),
    TestCase((2, [[1,0]], [[1,0]]), [True]),
    TestCase((2, [], [[1,0],[0,1]], ), [False, False]),
    TestCase((4, [[2,3],[2,1],[0,3],[0,1]], [[0,1],[0,3],[2,3],[3,0],[2,0],[0,2]]), [True,True,True,False,False,False])
]
Tester(SolutionDfs(), "checkIfPrerequisite")(test_cases)
Tester(SolutionDfsMemo(), "checkIfPrerequisite")(test_cases)
Tester(SolutionFloydWarshall(), "checkIfPrerequisite")(test_cases)
Tester(SolutionTopSort(), "checkIfPrerequisite")(test_cases)

Testing Report for Function: SolutionDfs.checkIfPrerequisite
Summary:
  Total Test Cases: 5
  Passed: 5
  Failed: 0
Testing Report for Function: SolutionDfsMemo.checkIfPrerequisite
Summary:
  Total Test Cases: 5
  Passed: 5
  Failed: 0
Testing Report for Function: SolutionFloydWarshall.checkIfPrerequisite
Summary:
  Total Test Cases: 5
  Passed: 5
  Failed: 0
[set(), {0}]
[set(), set()]
[{1}, set()]
[set(), set()]
[set(), {0, 2}, set(), {0, 2}]
Testing Report for Function: SolutionTopSort.checkIfPrerequisite
Summary:
  Total Test Cases: 5
  Passed: 5
  Failed: 0
